In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from functools import reduce
%matplotlib inline

In [2]:
#need to make sure you keep all of your source data close at hand so you can cite your sources....

#census election data from here:
#https://www.census.gov/data/tables/2012/demo/voting-and-registration/p20-568.html

reading in my data on elections from 1976..each table represents data for the named elections from the years in the comments

In [3]:
#I deleted these tables from my data folder so these cells reading in the house, 
#senate and pres data will no longer work.  I deleted them because my project became focused on TX 
#as I drilled down.  These tables are only at a national level divided into state returns.  

#US House 1976-2018
#house = pd.read_csv('../data/1976-2018-house3.csv')

In [4]:
# house.info()

In [5]:
# house.head()

In [6]:
# house.tail()

In [7]:
#US Senate 1976-2020
#senate = pd.read_csv('../data/1976-2020-senate.csv')

In [8]:
# senate.info()

In [9]:
# senate.head()

In [10]:
# senate.tail()

In [11]:
#US president 1976-2020
#president = pd.read_csv('../data/1976-2020-president.csv')

In [12]:
# president.info()

In [13]:
# president.head()

In [14]:
# president.tail()

Tables below were not deleted


In [15]:
#county returns president 2000-2016
county_pres = pd.read_csv('../data/countypres_2000-2016.csv')

In [16]:
county_pres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50524 entries, 0 to 50523
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            50524 non-null  int64  
 1   state           50524 non-null  object 
 2   state_po        50460 non-null  object 
 3   county          50524 non-null  object 
 4   FIPS            50476 non-null  float64
 5   office          50524 non-null  object 
 6   candidate       50524 non-null  object 
 7   party           34735 non-null  object 
 8   candidatevotes  50120 non-null  float64
 9   totalvotes      50524 non-null  int64  
 10  version         50524 non-null  int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 4.2+ MB


In [17]:
county_pres.head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2000,Alabama,AL,Autauga,1001.0,President,Al Gore,democrat,4942.0,17208,20191203
1,2000,Alabama,AL,Autauga,1001.0,President,George W. Bush,republican,11993.0,17208,20191203
2,2000,Alabama,AL,Autauga,1001.0,President,Ralph Nader,green,160.0,17208,20191203
3,2000,Alabama,AL,Autauga,1001.0,President,Other,NaN,113.0,17208,20191203
4,2000,Alabama,AL,Baldwin,1003.0,President,Al Gore,democrat,13997.0,56480,20191203


In [18]:
#county_pres.tail()

Cleaning up the presidential election result table:

In [19]:
tx_county_pres = county_pres[county_pres['state_po'] == 'TX']
tx_county_pres.head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
9972,2000,Texas,TX,Anderson,48001.0,President,Al Gore,democrat,5041.0,15080,20191203
9973,2000,Texas,TX,Anderson,48001.0,President,George W. Bush,republican,9835.0,15080,20191203
9974,2000,Texas,TX,Anderson,48001.0,President,Ralph Nader,green,131.0,15080,20191203
9975,2000,Texas,TX,Anderson,48001.0,President,Other,NaN,73.0,15080,20191203
9976,2000,Texas,TX,Andrews,48003.0,President,Al Gore,democrat,876.0,4024,20191203


In [20]:
#cleaning up this table by removing unnecessary columns
#I came back on 6/23 to put back a few more columns I thought might be useful for my story

tx_county_pres = tx_county_pres[['year','county','party','candidatevotes','totalvotes']].reset_index(drop = True)
#used below code for index reset and then realized I could just nest the code above.  :)
#tx_cnty_pres = tx_county_pres.reset_index(drop = True)

In [21]:

tx_cnty_pres = tx_county_pres.drop_duplicates()
tx_cnty_pres['county'] = tx_cnty_pres['county'].str.upper()
#thought I might need to ad "county" to this one but really the better thing to do 
#is drop it from the other...
#tx_cnty_pres['county'] = tx_county_pres['county'].apply(lambda x: f"{x} County")

In [22]:
tx_cnty_pres.count()

year              4064
county            4064
party             2794
candidatevotes    4064
totalvotes        4064
dtype: int64

In [23]:
tx_cnty_pres.head()

,year,county,party,candidatevotes,totalvotes
0,2000,ANDERSON,democrat,5041.0,15080
1,2000,ANDERSON,republican,9835.0,15080
2,2000,ANDERSON,green,131.0,15080
3,2000,ANDERSON,NaN,73.0,15080
4,2000,ANDREWS,democrat,876.0,4024


In [24]:
#I tried to clean this in Python and it wasn't working correctly (the code worked but tableau was not showing the filtered dataframe).  
#Certain it is a user error problem but I went ahead and used Tableau to filter out the 'green' and NaN values from this dataframe

tx_cnty_pres = tx_county_pres.dropna()
tx_cnty_pres = tx_cnty_pres[tx_county_pres.party != 'green']
tx_cnty_pres.head()

<ipython-input-24-26108f7089fe>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tx_cnty_pres = tx_cnty_pres[tx_county_pres.party != 'green']


,year,county,party,candidatevotes,totalvotes
0,2000,Anderson,democrat,5041.0,15080
1,2000,Anderson,republican,9835.0,15080
4,2000,Andrews,democrat,876.0,4024
5,2000,Andrews,republican,3091.0,4024
8,2000,Angelina,democrat,9957.0,26974


In [25]:
democrat = tx_cnty_pres[tx_cnty_pres['party'] == 'democrat']
republican = tx_cnty_pres[tx_cnty_pres['party'] == 'republican'] 

In [26]:
party_merged = pd.merge(democrat, republican, how = 'left', on = ['county','year'])
party_merged['county'] = party_merged['county'].str.upper()
party_merged.head()

,year,county,party_x,candidatevotes_x,totalvotes_x,party_y,candidatevotes_y,totalvotes_y
0,2000,ANDERSON,democrat,5041.0,15080,republican,9835.0,15080
1,2000,ANDREWS,democrat,876.0,4024,republican,3091.0,4024
2,2000,ANGELINA,democrat,9957.0,26974,republican,16648.0,26974
3,2000,ARANSAS,democrat,2637.0,8247,republican,5390.0,8247
4,2000,ARCHER,democrat,993.0,3997,republican,2951.0,3997


In [27]:
#tx_cnty_pres['state'] = 'TX'

In [28]:
#tx_cnty_pres.year = tx_cnty_pres.year.astype('datetime64[ns]')
tx_cnty_pres.head()


,year,county,party,candidatevotes,totalvotes
0,2000,Anderson,democrat,5041.0,15080
1,2000,Anderson,republican,9835.0,15080
4,2000,Andrews,democrat,876.0,4024
5,2000,Andrews,republican,3091.0,4024
8,2000,Angelina,democrat,9957.0,26974


In [29]:

tx_cnty_pres.tail()

,year,county,party,candidatevotes,totalvotes
4056,2016,Young,republican,6601.0,7707
4058,2016,Zapata,democrat,2063.0,3142
4059,2016,Zapata,republican,1029.0,3142
4061,2016,Zavala,democrat,2636.0,3396
4062,2016,Zavala,republican,694.0,3396


Moving this dataframe to a csv:

In [30]:
tx_cnty_pres.to_csv('../data/tx_cnty_pres.csv')

reading in census data for elections by year

In [31]:
reg_sex_state_2010 = pd.read_excel('../data/registration_race_sex_state_2010.xls',header = 2)

In [32]:
#this is going to need some cleaning....cool.
#reg_sex_state_2010.info()

In [33]:
#population of 18yo +in thousands 

# reg_sex_state_2010.head(15)

In [34]:
#bottom of table contains some (un)helpful info that should have been in a readme file. 

# reg_sex_state_2010.tail(100)

In [35]:
#tx voting population 18years+

tx_reg_2010 = pd.concat([reg_sex_state_2010.iloc[:11,:],reg_sex_state_2010.iloc[484:495,:]],axis = 0)

I messed around with the code below trying to set the header...until I found an argument to put into the read_excel function...which I thought worked like a dream but it's currently not working the way I want every time I load it...must be something with my notebook...and specifically it's the fact that I wasn't giving the DF a new variable name.  So yes, technically it was the notebook but really it was because of a user decision.

In [36]:


#tx voting population 18years+

# tx_reg_2010.columns = tx_reg_2010.iloc[1]
# tx_reg_clean = tx_reg_2010.drop(tx_reg_2010.index[1], axis = 0)

# tx_reg_2010.drop(tx_reg_2010.iloc[:1,:], axis = 0)

In [37]:
tx_reg_2010 = tx_reg_2010.rename(columns = {'Unnamed: 1': 'Race/Gender'})

In [38]:
#tx_reg_2010.info()

Turns out this tx_reg_2010 is not what I need to tell my story...am commenting out the display code for it so i don't have to wait for it to load.


In [39]:
#This needs to be cleaned A LOT still....
#also I need to read in data for all of the years after this up to at least 2016. yikes...need a function maybe

#tx_reg_2010


In [40]:
###Not sure I'm going to use all of above but I have it....


reading in TX County Census data 2010-2019

first age data:

In [41]:
tx_census_age = pd.read_csv('../data/TX_county_census_data.csv')

In [42]:
# tx_census_age.info()

In [43]:
# tx_census_age.head()

In [44]:
# tx_census_age.tail()

In [45]:
#tried all of the below to drop columns which didn't work but I finally found something that did...

tx_age = pd.concat([tx_census_age.iloc[:,:9],tx_census_age.iloc[:,18:21],tx_census_age.iloc[:,24:27],
                  tx_census_age.iloc[:,36:39],tx_census_age.iloc[:,90:]],axis = 1)

# tx_census_age.drop(tx_census_age.columns[[1]], axis = 1)
# tx_age = tx_census_age.drop(list(range(9,19)), axis = 1)

# tx_census_age.drop(tx_census_age.iloc[:,21:23], axis = 1)
# tx_census_age.drop(tx_census_age.iloc[:,28:35], axis = 1)
# tx_census_age.drop(tx_census_age.iloc[:,38:77], inplace = True, axis = 1)

In [46]:
#tx_age.info()

In [47]:
# tx_age

reading TX census race data

In [48]:
tx_census_race = pd.read_csv('../data/TX_age_sex_race.csv')

For the YEAR and AGEGRP columns of this DF 
there are different variables...explained below

I'm including these for my own use:


The key for the YEAR variable is as follows:
1 = 4/1/2010 Census population
2 = 4/1/2010 population estimates base
3 = 7/1/2010 population estimate
4 = 7/1/2011 population estimate
5 = 7/1/2012 population estimate
6 = 7/1/2013 population estimate
7 = 7/1/2014 population estimate
8 = 7/1/2015 population estimate
9 = 7/1/2016 population estimate
10 = 7/1/2017 population estimate
11 = 7/1/2018 population estimate
12 = 7/1/2019 population estimate


The key for AGEGRP is as follows:
0 = Total
1 = Age 0 to 4 years
2 = Age 5 to 9 years
3 = Age 10 to 14 years
4 = Age 15 to 19 years
5 = Age 20 to 24 years
6 = Age 25 to 29 years
7 = Age 30 to 34 years
8 = Age 35 to 39 years
9 = Age 40 to 44 years
10 = Age 45 to 49 years
11 = Age 50 to 54 years
12 = Age 55 to 59 years
13 = Age 60 to 64 years
14 = Age 65 to 69 years
15 = Age 70 to 74 years
16 = Age 75 to 79 years
17 = Age 80 to 84 years
18 = Age 85 years or older

In [49]:
#tx_census_race.info()

In [50]:
# tx_census_race.head(20)

In [51]:
# tx_census_race.tail()

In [52]:
#creating a table of the census race data I think I need:
tx_race = pd.concat([tx_census_race.iloc[:,:18],tx_census_race.iloc[:,24:26],tx_census_race.iloc[:,28:30], 
                     tx_census_race.iloc[:,34:36],  tx_census_race.iloc[:,56:58]],axis = 1)
                  


In [53]:
#tx_race.info()

In [54]:
# for index, row in tx_race.iterrows():
#     if row['AGEGRP'] > 0:
#         tx_race.drop(index, inplace=True)

#trying to remove all agegrp but the total
#and remoce YEAR with value of 1 or 2 so the data would be even distributed over 12 months. 
# July 1, 2010 to July 1, 2019
tx_race = tx_race[tx_race.AGEGRP == 0]
tx_race = tx_race[tx_race.YEAR > 2]

Again, from above:

The key for the YEAR variable is as follows: 1 = 4/1/2010 Census population 2 = 4/1/2010 population estimates base 3 = 7/1/2010 population estimate 4 = 7/1/2011 population estimate 5 = 7/1/2012 population estimate 6 = 7/1/2013 population estimate 7 = 7/1/2014 population estimate 8 = 7/1/2015 population estimate 9 = 7/1/2016 population estimate 10 = 7/1/2017 population estimate 11 = 7/1/2018 population estimate 12 = 7/1/2019 population estimate

In [55]:
#need a loop to make the year column show actual year...not the 
#classification from the documentation of the csv
#nope...
tx_race['YEAR'] = tx_race['YEAR'] + 2007

In [56]:
tx_race.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,AA_MALE,AA_FEMALE,BAC_MALE,BAC_FEMALE,AAC_MALE,AAC_FEMALE,NHWA_MALE,NHWA_FEMALE,H_MALE,H_FEMALE
38,50,48,1,Texas,Anderson County,2010,0,58493,35542,22951,...,249,176,9367,3660,361,230,19740,16095,6391,2949
57,50,48,1,Texas,Anderson County,2011,0,58394,35586,22808,...,255,178,9449,3634,374,237,19634,15933,6438,2990
76,50,48,1,Texas,Anderson County,2012,0,58059,35327,22732,...,273,190,9389,3575,403,242,19354,15853,6506,3047
95,50,48,1,Texas,Anderson County,2013,0,57960,35315,22645,...,274,184,9356,3580,404,249,19291,15695,6556,3121
114,50,48,1,Texas,Anderson County,2014,0,57829,35350,22479,...,275,166,9468,3573,408,221,19187,15506,6621,3180


In [57]:
# tx_race.tail()

In [58]:
#Thinking that I might need to use the not-hispanic white alone numbers to get what I'm looking for....
#NHWA_MALE Not Hispanic, White alone male population
#NHWA_FEMALE Not Hispanic, White alone female population
#used this code first to get the race measure I thought I needed.  

#top code is to find total number of people in each county per reporting year that identify as white alone(includes hispanic)
#tx_race['WA_POP'] = tx_race['WA_MALE'] + tx_race['WA_FEMALE']

#this code is total number of people in each county per reporting year that identify as white alone (NOT Hispanic)
tx_race['NHWA_POP'] = tx_race['NHWA_MALE'] + tx_race['NHWA_FEMALE']
#I will only need the second line but am leaving the first one in to show how I got here...
#was looking for counties with majority not white to compare with counties with Majority white

In [59]:
#tx_race.head()

In [60]:
#trying to come up with a measure of people in each county per reporting year that do not identify as "white alone" 
#came up with this first before I realized my previous error in not understanding the categories in the data
#tx_race['NW_POP'] = tx_race['TOT_POP'] - tx_race['WA_POP']

#2nd line is dropping word 'County' from the CNTYNAME colunn

tx_race['NOT_ANGLO_POP'] = tx_race['TOT_POP'] - tx_race['NHWA_POP']
tx_race['CTYNAME'] = tx_race['CTYNAME'].str.replace('County','')

In [61]:
#tx_race.head()

I initially created two tables to show White(Anglo) and non-white majorities in counties
If I'm going to use them on a map I realized that I maybe should keep them together...
so here is the table above reduced down to the columns I want: 

In [62]:
tx_race_cnty = tx_race[['CTYNAME', 'YEAR', 'TOT_POP', 'NHWA_POP','NOT_ANGLO_POP']]

In [101]:

#adding a matching column of 'state' to try powerBI
#tx_race_cnty['state'] = 'TX'
tx_race_cnty.head(20)

,county,year,TOT_POP,NHWA_POP,NOT_ANGLO_POP,State
38,ANDERSON,2010,58493,35835,22658,TX
57,ANDERSON,2011,58394,35567,22827,TX
76,ANDERSON,2012,58059,35207,22852,TX
95,ANDERSON,2013,57960,34986,22974,TX
114,ANDERSON,2014,57829,34693,23136,TX
133,ANDERSON,2015,57639,34336,23303,TX
152,ANDERSON,2016,57526,34218,23308,TX
171,ANDERSON,2017,58175,34256,23919,TX
190,ANDERSON,2018,57974,33950,24024,TX
209,ANDERSON,2019,57735,33638,24097,TX


In [64]:
#tx_race_cnty['CTYNAME'] = tx_race_cnty['CTYNAME'].str.replace('County','')
tx_race_cnty.head(20)

,CTYNAME,YEAR,TOT_POP,NHWA_POP,NOT_ANGLO_POP
38,Anderson,2010,58493,35835,22658
57,Anderson,2011,58394,35567,22827
76,Anderson,2012,58059,35207,22852
95,Anderson,2013,57960,34986,22974
114,Anderson,2014,57829,34693,23136
133,Anderson,2015,57639,34336,23303
152,Anderson,2016,57526,34218,23308
171,Anderson,2017,58175,34256,23919
190,Anderson,2018,57974,33950,24024
209,Anderson,2019,57735,33638,24097


In [65]:


#sent DF to a csv so my viz notebook can use it but afterwards realized I need to clean the table a little more maybe...
tx_race_cnty = tx_race_cnty.rename({'CTYNAME': 'county','YEAR':'year'}, axis = 1)
tx_race_cnty.head()

,county,year,TOT_POP,NHWA_POP,NOT_ANGLO_POP
38,Anderson,2010,58493,35835,22658
57,Anderson,2011,58394,35567,22827
76,Anderson,2012,58059,35207,22852
95,Anderson,2013,57960,34986,22974
114,Anderson,2014,57829,34693,23136


In [66]:

tx_race_cnty['State'] = 'TX'
tx_race_cnty['county'] = tx_race_cnty['county'].str.upper()


tx_race_cnty.to_csv('../data/tx_race_cnty.csv')

In [67]:
tx_race_cnty1 = tx_race[tx_race.NHWA_POP < tx_race.NOT_ANGLO_POP]
tx_race_cnty2 = tx_race[tx_race.NHWA_POP > tx_race.NOT_ANGLO_POP]

In [68]:
tx_race_cnty_not_anglo = tx_race_cnty1[[ 'CTYNAME', 'YEAR', 'TOT_POP', 'NHWA_POP','NOT_ANGLO_POP']]
tx_race_cnty_anglo = tx_race_cnty2[[ 'CTYNAME', 'YEAR', 'TOT_POP', 'NHWA_POP','NOT_ANGLO_POP']]

In [69]:
#(tx_race.NHWA_POP < tx_race.NOT_ANGLO_POP).sum()


In [70]:
# here is my new dataframe representing counties that are majority not Anglo
#below code is giving me a warning so I'm putting the removal of 'county' on hold for now
#tx_race_cnty_not_anglo['CTYNAME'] = tx_race_cnty_not_anglo['CTYNAME'].str.replace('County','', regex = True)

In [71]:
#tx_race_cnty_not_anglo.head()

In [72]:
#counties with majority anglo population
#tx_race_cnty_anglo.head()

In [73]:
#Need to use the below code to save these dataframes to .csv so I can use with a map
#df.to_csv(r'C:\Users\Admin\Desktop\file3.csv', index=False)
#Might need to go back in the above code and make a stripped down version of the DF with all counties present
#will be easier to use that on a map maybe

I have TX registration data from 2010 to 2020 that is clean but each csv lacks a year column.  I'm going to add a column to each 
Feel like I could have made a loop for this somehow but there was no time to mess with that. 

In [74]:
reg_2010 =  pd.read_csv('../data/reg data/reg_2010.csv')
reg_2011 =  pd.read_csv('../data/reg data/reg_2011.csv')
reg_2012 =  pd.read_csv('../data/reg data/reg_2012.csv')
reg_2013 =  pd.read_csv('../data/reg data/reg_2013.csv')
reg_2014 =  pd.read_csv('../data/reg data/reg_2014.csv')
reg_2015 =  pd.read_csv('../data/reg data/reg_2015.csv')
reg_2016 =  pd.read_csv('../data/reg data/reg_2016.csv')
reg_2017 =  pd.read_csv('../data/reg data/reg_2017.csv')
reg_2018 =  pd.read_csv('../data/reg data/reg_2018.csv')
reg_2019 =  pd.read_csv('../data/reg data/reg_2019.csv')

In [75]:
#reg_2010.head()

In [76]:
reg_2010.insert(1,'Year', 2010)
reg_2011.insert(1,'Year', 2011)
reg_2012.insert(1,'Year', 2012)
reg_2013.insert(1,'Year', 2013)
reg_2014.insert(1,'Year', 2014)
reg_2015.insert(1,'Year', 2015)
reg_2016.insert(1,'Year', 2016)
reg_2017.insert(1,'Year', 2017)
reg_2018.insert(1,'Year', 2018)
reg_2019.insert(1,'Year', 2019)

In [77]:
#reg_2010.head()

In [78]:
#reg_2011.head()

moving all of these updated dfs to a new data folder:

In [79]:
reg_2010.to_csv('../data/reg_with_year/reg_2010.csv')
reg_2011.to_csv('../data/reg_with_year/reg_2011.csv')
reg_2012.to_csv('../data/reg_with_year/reg_2012.csv')
reg_2013.to_csv('../data/reg_with_year/reg_2013.csv')
reg_2014.to_csv('../data/reg_with_year/reg_2014.csv')
reg_2015.to_csv('../data/reg_with_year/reg_2015.csv')
reg_2016.to_csv('../data/reg_with_year/reg_2016.csv')
reg_2017.to_csv('../data/reg_with_year/reg_2017.csv')
reg_2018.to_csv('../data/reg_with_year/reg_2018.csv')
reg_2019.to_csv('../data/reg_with_year/reg_2019.csv')

In [80]:
tx_reg = pd.concat([reg_2010,reg_2011,reg_2012,reg_2013,reg_2014,reg_2015,reg_2016,reg_2017,reg_2018,reg_2019], axis = 0)

In [81]:
#tx_reg.tail()

In [82]:
#tx_reg.info()

In [83]:
#tx_reg.Year = tx_reg.Year.astype('datetime64[ns]')
#tx_reg = pd.to_datetime(tx_reg.Year, unit = 's')
#tx_reg.info()

In [84]:
#  coming back into this DF to add a state column in hopes of getting power BI to work for me. 
#tx_reg['state'] = 'TX'


In [85]:
tx_reg.head()

,County Name,Year,Precincts,Voter Registration,Suspense Voters,Non-Suspense Voters
0,ANDERSON,2010,24,"26,298","3,668","22,630"
1,ANDREWS,2010,4,"8,212","1,127","7,085"
2,ANGELINA,2010,46,"46,456","6,065","40,391"
3,ARANSAS,2010,6,"15,355","2,381","12,974"
4,ARCHER,2010,13,"6,474",864,"5,610"


In [86]:

tx_reg = tx_reg.rename({'County Name': 'county', 'Year':'year'}, axis = 1)

In [87]:

tx_race_cnty['State'] = 'TX'
tx_race_cnty['county'] = tx_race_cnty['county'].str.upper()
tx_race_cnty.head()

,county,year,TOT_POP,NHWA_POP,NOT_ANGLO_POP,State
38,ANDERSON,2010,58493,35835,22658,TX
57,ANDERSON,2011,58394,35567,22827,TX
76,ANDERSON,2012,58059,35207,22852,TX
95,ANDERSON,2013,57960,34986,22974,TX
114,ANDERSON,2014,57829,34693,23136,TX


In [88]:
tx_reg.head()

,county,year,Precincts,Voter Registration,Suspense Voters,Non-Suspense Voters
0,ANDERSON,2010,24,"26,298","3,668","22,630"
1,ANDREWS,2010,4,"8,212","1,127","7,085"
2,ANGELINA,2010,46,"46,456","6,065","40,391"
3,ARANSAS,2010,6,"15,355","2,381","12,974"
4,ARCHER,2010,13,"6,474",864,"5,610"


In [89]:
tx_reg.to_csv('../data/reg_with_year/tx_reg.csv')

In [90]:
# an attempt to merge my three necessary DFs:

#tx_data_merged = tx_race_cnty.merge(tx_reg.merge(tx_cnty_pres, on ='county'), on = 'county')
#df1.merge(df2.merge(df3, on='date'), on='date')

In [91]:
#tx_data_merged.head()

In [92]:
#first_merge = tx_race_cnty.merge(tx_reg, on = ['county','Year'], how = 'left')

In [93]:
#first_merge.info()

In [94]:
#second_merge = first_merge.merge(tx_reg, on = 'county')

In [95]:
#second_merge.head()

In [96]:
data_frames = [tx_reg, tx_cnty_pres]
              

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['county'],how='outer'), data_frames)
#df_merged = df_merged.sort_values(['Year'], ascending = (False))

In [97]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5080 entries, 0 to 5079
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   county               5080 non-null   object 
 1   year_x               2540 non-null   float64
 2   Precincts            2540 non-null   object 
 3   Voter Registration   2540 non-null   object 
 4   Suspense Voters      2540 non-null   object 
 5   Non-Suspense Voters  2540 non-null   object 
 6   year_y               2540 non-null   float64
 7   party                2540 non-null   object 
 8   candidatevotes       2540 non-null   float64
 9   totalvotes           2540 non-null   float64
dtypes: float64(4), object(6)
memory usage: 436.6+ KB


In [98]:
df_merged.head(20)

,county,year_x,Precincts,Voter Registration,Suspense Voters,Non-Suspense Voters,year_y,party,candidatevotes,totalvotes
0,ANDERSON,2010.0,24,"26,298","3,668","22,630",NaN,NaN,NaN,NaN
1,ANDERSON,2011.0,24,"25,297","1,716","23,581",NaN,NaN,NaN,NaN
2,ANDERSON,2012.0,24,"25,821","1,894","23,927",NaN,NaN,NaN,NaN
3,ANDERSON,2013.0,24,"25,711","1,634","24,077",NaN,NaN,NaN,NaN
4,ANDERSON,2014.0,23,"26,332","3,543","22,789",NaN,NaN,NaN,NaN
5,ANDERSON,2015.0,24,"26,090","1,937","24,153",NaN,NaN,NaN,NaN
6,ANDERSON,2016.0,24,"26,773","3,864","22,909",NaN,NaN,NaN,NaN
7,ANDERSON,2017.0,24,"28,242","2,887","25,355",NaN,NaN,NaN,NaN
8,ANDERSON,2018.0,24,"27,788","3,993","23,795",NaN,NaN,NaN,NaN
9,ANDERSON,2019.0,22,"27,491","1,826","25,665",NaN,NaN,NaN,NaN


In [99]:
anderson = df_merged[df_merged['county'] == 'ANDERSON']

In [100]:
anderson

,county,year_x,Precincts,Voter Registration,Suspense Voters,Non-Suspense Voters,year_y,party,candidatevotes,totalvotes
0,ANDERSON,2010.0,24,"26,298","3,668","22,630",NaN,NaN,NaN,NaN
1,ANDERSON,2011.0,24,"25,297","1,716","23,581",NaN,NaN,NaN,NaN
2,ANDERSON,2012.0,24,"25,821","1,894","23,927",NaN,NaN,NaN,NaN
3,ANDERSON,2013.0,24,"25,711","1,634","24,077",NaN,NaN,NaN,NaN
4,ANDERSON,2014.0,23,"26,332","3,543","22,789",NaN,NaN,NaN,NaN
5,ANDERSON,2015.0,24,"26,090","1,937","24,153",NaN,NaN,NaN,NaN
6,ANDERSON,2016.0,24,"26,773","3,864","22,909",NaN,NaN,NaN,NaN
7,ANDERSON,2017.0,24,"28,242","2,887","25,355",NaN,NaN,NaN,NaN
8,ANDERSON,2018.0,24,"27,788","3,993","23,795",NaN,NaN,NaN,NaN
9,ANDERSON,2019.0,22,"27,491","1,826","25,665",NaN,NaN,NaN,NaN
